In [109]:
import pandas as pd
from pathlib import Path


___
Remove outliers

In [110]:

dir_fcgr = "../data/fcgr/6mer"
path_outliers = "../experiments-paper/6mer/07_25_2024-autoencoder/cross-validation/outliers/outliers.csv"
path_save = "../experiments-paper/6mer/07_25_2024-autoencoder/list_train.txt"

In [111]:
list_all = [str(p)[3:] for p in Path(dir_fcgr).rglob("*npy")]


In [112]:
len(list_all)

657775

In [113]:
df_outliers = pd.read_csv(path_outliers, sep="\t")
outliers = df_outliers.path.tolist()

In [114]:
len(outliers)

7373

In [115]:
list_all[:2]

['data/fcgr/6mer/streptococcus_pneumoniae__05/SAMEA2358933.npy',
 'data/fcgr/6mer/streptococcus_pneumoniae__05/SAMEA2612794.npy']

In [116]:
outliers[:2]

['data/fcgr/6mer/dustbin__10/SAMN02744650.npy',
 'data/fcgr/6mer/dustbin__21/SAMN02801811.npy']

In [117]:
list_no_outliers = list(set(list_all)-set(outliers))
len(list_no_outliers)

650402

___
Fix labels

In [118]:
path_ani = "../experiments-paper/6mer/07_25_2024-autoencoder/cross-validation/confident-learning/ani.tsv"

In [119]:
ani = pd.read_csv(path_ani, sep="\t")

In [120]:
ani.head()

,Unnamed: 0,path_ebi,path_ref,ani,3,4,label_ref,sample_id,label_ebi
0,0,experiments-paper/6mer/07_25_2024-autoencoder/...,data/ref-seqs/GCF_000008445.1/ncbi_dataset/dat...,97.5367,1430,1623,bacillus_anthracis,SAMEA41934418,bacillus_cereus
1,1,experiments-paper/6mer/07_25_2024-autoencoder/...,data/ref-seqs/GCF_002220285.1/ncbi_dataset/dat...,91.4518,1349,1623,bacillus_cereus,SAMEA41934418,bacillus_cereus
2,2,experiments-paper/6mer/07_25_2024-autoencoder/...,data/ref-seqs/GCF_001888925.1/ncbi_dataset/dat...,82.1911,379,638,lactobacillus_delbrueckii,SAMN02369379,lactobacillus_delbrueckii
3,3,experiments-paper/6mer/07_25_2024-autoencoder/...,data/ref-seqs/GCF_000008865.2/ncbi_dataset/dat...,97.5226,1402,2541,escherichia_coli,SAMN06018941,escherichia_coli
4,4,experiments-paper/6mer/07_25_2024-autoencoder/...,data/ref-seqs/GCF_000006945.2/ncbi_dataset/dat...,82.0419,917,2541,salmonella_enterica,SAMN06018941,escherichia_coli


In [121]:
path_labels = "../data/labels_krakenbracken_by_sampleid.txt"

In [122]:
labels_by_sampleid = dict()
with open(path_labels, "r") as fp:
    for line in fp.readlines():
        try:
            # TODO: any better way to standarize the label? could change for other experiments
            sample_id, label = line.replace("\n","").strip().split("\t")
            sample_id = sample_id.strip()
            label = "_".join(label.lower().strip().split(" "))
            labels_by_sampleid[sample_id] = label
        except:
            continue

In [123]:
# update modified labels
for d in ani.query("ani >= 95.0").to_dict("records"):
    new_label = d["label_ref"]
    sample_id = d["sample_id"]
    labels_by_sampleid[sample_id] = new_label

Save not outliers with their labels for creating the index

In [125]:
Path(path_save).parent.mkdir(exist_ok=True, parents=True)
with open(path_save, "w") as fp:
    for path in list_no_outliers:
        try:
            sample_id = Path(path).stem
            label = labels_by_sampleid[sample_id]
            fp.write(f"{path}\t{label}\n")
        except:
            continue

In [ ]:
new_label

'enterobacter_hormaechei'

In [ ]:
labels_by_sampleid

{'sample_id': 'label',
 'SAMD00000344': 'lactobacillus_hokkaidonensis',
 'SAMD00000345': 'xanthomonas_euvesicatoria',
 'SAMD00000355': 'streptococcus_pyogenes',
 'SAMD00000550': 'corynebacterium_glutamicum',
 'SAMD00000552': 'corynebacterium_glutamicum',
 'SAMD00000562': 'vibrio_parahaemolyticus',
 'SAMD00000572': 'mycobacteroides_chelonae',
 'SAMD00000584': 'cronobacter_sakazakii',
 'SAMD00000585': 'serratia_ficaria',
 'SAMD00000605': 'pandoraea_oxalativorans',
 'SAMD00000611': 'staphylococcus_epidermidis',
 'SAMD00000612': 'clostridium_perfringens',
 'SAMD00000644': 'mycobacterium_chimaera',
 'SAMD00000649': 'escherichia_coli',
 'SAMD00000652': 'staphylococcus_aureus',
 'SAMD00000665': 'bacillus_subtilis',
 'SAMD00000666': 'bacillus_subtilis',
 'SAMD00000667': 'bacillus_subtilis',
 'SAMD00000669': 'elizabethkingia_meningoseptica',
 'SAMD00000692': 'vibrio_diabolicus',
 'SAMD00000699': 'escherichia_coli',
 'SAMD00000724': 'escherichia_coli',
 'SAMD00000754': 'escherichia_coli',
 'SAMD

In [ ]:
Path(list_no_outliers[0]).stem

'SAMN06091944'

In [ ]:
labels_by_sampleid["SAMN06091944"]

'mycobacterium_tuberculosis'